In [ ]:
import multiprocessing
from multiprocessing import Pool
from functools import partial
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
from sklearn.metrics import accuracy_score
import winsound
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import workers_1

In [1]:
# stored best initializations for X and different parametrisations:
%store -r pool_list

In [ ]:
# extract just initial x values
plist=np.array(pool_list)[:,1]

In [ ]:
#function for spliting X,Y,G into desired timesteps, the output of this function is the result of Gibbs sampling algorithm:
# returns probability which is the model estimate of X:
# this code is parallelized on different timesteps:
def Step_Gibbs_parallel(X):
    arg=[]
    pool_list=[]
    for i in range(1,5):
        time_step=i*90
        G_=G[:time_step]
        # here by passing Y related to YF_missing for specified value of mu, we can apply desired amount of missing value:
        Y_=Y[:,:time_step]
        X_=X[:,:time_step]
        pool_list.append([X_,G_,Y_])
        
    if __name__ ==  '__main__': 
        with Pool(processes =4) as pool:

            parallel_output = pool.starmap(workers_1.Step_Gibbs,pool_list )# use tqdm to show the progress
            pool.close()
            pool.join()
    return parallel_output

In [6]:
for i in range(1,5):
    print(i*90)

90
180
270
360


In [10]:
# function for plotting figure 1,outputs the array of AUC 
def plot_figure_1(Trained):
    roc_=[]
    for i in range(len(Trained)):
        for j in range(9):
            Train=np.array(Trained)[i][2][j]
            y_score=np.hstack(Train)
            y_test=np.hstack(X_true[:,:Train.shape[1]])
            roc_.append([(i*9+j)*10,plot_ROC(y_score,y_test)])
    plt.plot(roc_) 
    return  roc_

In [ ]:
if os.path.exists('Xreal.npy'):
    X_true = np.load('Xreal.npy')

In [ ]:
duration = 1000  # milliseconds
freq = 440  # Hz 
# initialize parameters for beta distributions:
a_alpha=1
b_alpha=400
a_beta=1
b_beta=800
a_betaf=8
b_betaf=800
a_gama=50
b_gama=200
a_teta0=10
b_teta0=1000
a_teta1=9000
b_teta1=300
P=1
U=1
K=1
J=1
n,T=100,365
epsln=0.001
hyper_params=np.array([a_alpha,b_alpha,a_beta,b_beta,a_betaf,b_betaf,a_gama,b_gama,a_teta0,b_teta0,a_teta1,b_teta1])

if os.path.exists('Greal.npy'):
    G = np.load('Greal.npy')
if os.path.exists('YFreal.npy'):
    YF = np.load('YFreal.npy')
if os.path.exists('Freal.npy'):
    F = np.load('Freal.npy')
if os.path.exists('YF_missingreal.npy'):
    YF_missing = np.load('YF_missingreal.npy')
if os.path.exists('YF_missing_1real.npy'):
    YF_missing1 = np.load('YF_missing_1real.npy')
if os.path.exists('Xreal.npy'):
    X_True = np.load('Xreal.npy')
n,T=X_True.shape[0],X_True.shape[1]
if os.path.exists('params.npy'):
    params = np.load('params.npy')

unique_rows = np.unique(F, axis=0)
#calculate y_missing:
Y=np.zeros((n,T))
for i in range(n):
    for j in range(unique_rows.shape[0]):
        if unique_rows[j,i]==1:
            Y[i,:]=YF[j,:]


In [11]:
# for is just because of different initializations for X:
for i in range(len(plist)):
    X=plist[i]
    Trained=Step_Gibbs_parallel(X)
    AUC_t=plot_figure_1(Trained)

NameError: name 'plist' is not defined

In [35]:
import numpy as np
a=[[[[0,1,2],[.1,.2,.3]],1],[[[0,1,2],[.1,.2,.3]],2],[[[0,1,2],[.1,.2,.3]],3]]

In [54]:
print(a)

[[[[0, 1, 2], [0.1, 0.2, 0.3]], 1], [[[0, 1, 2], [0.1, 0.2, 0.3]], 2], [[[0, 1, 2], [0.1, 0.2, 0.3]], 3]]


In [55]:
a[0][0][0]

[0, 1, 2]

In [52]:
AUC=[]
for i in range(len(a)):
    AUC.append(    

In [50]:
np.mean(AUC,axis=1)

array([0.2, 0.2, 0.2])

In [ ]:
# to obtain AUC_t with regard to different mu, so here for is because of different values of mu:
%store -r MissingData
mu=MissingData[0]
for i in len(mu):
    YF_missing=MissingData[1]
    #calculate y_missing:
    Y=np.zeros((n,T))
    for i in range(n):
        for j in range(unique_rows.shape[0]):
            if unique_rows[j,i]==1:
                Y[i,:]=YF_missing[j,:]
    Trained=Step_Gibbs_parallel(X)
    AUC_t=plot_figure_1(Trained)
    result.append([AUC_t,mu[i]])
# avraging AUC's over mu, and report the result for each timestep:
AUC=[]
for i in range(len(result)):
    AUC.append(result[i][0][1])
AUC_avg=np.mean(AUC,axis=1) 
# plot  avarge value of AUC over mu  against t :
plt.plot(AUC[0][0][0],AUC_avg)

In [12]:
def plot_ROC(y_score,y_test):
    plt.figure()
    lw = 2
    #y_score=np.hstack(Train[2][j])
    #y_test=np.hstack(X)
    fpr, tpr, thresholds = roc_curve(y_test, y_score,pos_label=1)
    roc_auc = auc(fpr, tpr)
    plt.plot(
        fpr,
        tpr,
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % roc_auc,
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC for family test result problem")
    plt.legend(loc="lower right")
    plt.show()
    return roc_auc

In [ ]:
import multiprocessing
from multiprocessing import Pool
from functools import partial
from tqdm import tqdm
import numpy as np
import workers_1
if __name__ ==  '__main__': 
    with Pool(processes =4) as pool:

        parallel_output = pool.map(workers_1.Step_Gibbs_paralel,list(plist) )# use tqdm to show the progress
        pool.close()
        pool.join()
    
   